# Structure gemini outputs in a robust way

In [1]:
from google import genai


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """
)
print(response.text)

Get ready to slurp, savor, and spice things up! Here are a few iconic Asian soups, each with a delicious description and a list of their essential ingredients:

---

### 1. Vietnamese Pho Bo (Beef Noodle Soup)

**A Yummy Description:** Imagine a deeply aromatic, crystal-clear beef broth, simmered for hours with charred ginger, onions, star anise, cinnamon, and other warming spices. It’s ladled over silky-smooth rice noodles, topped with whisper-thin slices of rare beef that cook perfectly in the hot broth, and garnished generously with a vibrant bouquet of fresh herbs like Thai basil, cilantro, and mint, along with crunchy bean sprouts, lime wedges, and a fiery sliced chili. Each spoonful is a harmonious blend of savory, sweet, herbaceous, and spicy – a true comfort in a bowl.

**Ingredients:**

*   **For the Broth:**
    *   Beef bones (marrow bones, knuckle bones)
    *   Oxtail (optional, for richness)
    *   Onions
    *   Ginger
    *   Star anise
    *   Cinnamon sticks
    *   

In [2]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients. Structure it in json format
    """
)
print(response.text)

```json
[
  {
    "name": "Tom Yum Goong",
    "country_of_origin": "Thailand",
    "description": "A vibrant and exhilarating Thai hot and sour soup, bursting with aromatic herbs and a spicy kick! Imagine plump, succulent shrimp swimming in a fragrant broth infused with lemongrass, galangal, kaffir lime leaves, and a fiery chili paste, balanced perfectly with tangy lime juice. It's a symphony of flavors that will awaken your senses and leave you craving more.",
    "ingredients": [
      "Shrimp (peeled and deveined)",
      "Mushrooms (straw mushrooms or oyster mushrooms)",
      "Lemongrass (bruised and sliced)",
      "Galangal (sliced)",
      "Kaffir lime leaves (torn)",
      "Thai chilies (pounded or sliced)",
      "Fish sauce",
      "Lime juice",
      "Tom Yum paste (optional, for intense flavor)",
      "Chicken or vegetable broth",
      "Cilantro (for garnish)",
      "Evaporated milk or coconut milk (for creamy version, 'Tom Yum Goong Nam Khon')"
    ]
  },
  {
    "nam

In [3]:
from pydantic import BaseModel
from google import genai


class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]},
)

print(response.text)

[{"recipe_name":"Miso Soup","description":"A traditional Japanese soup consisting of a dashi stock into which softened miso paste is mixed. Various optional ingredients may be added depending on the region and season, such as tofu, wakame seaweed, and green onions. It's known for its umami-rich flavor and comforting warmth.","ingredients":["Dashi stock","Miso paste","Silken tofu","Wakame seaweed","Green onions"]},{"recipe_name":"Tom Yum Goong","description":"A vibrant and aromatic hot and sour soup from Thailand, typically made with shrimp. It's characterized by its distinct blend of spicy chili, sour lime, fragrant lemongrass, galangal, and kaffir lime leaves. A truly invigorating and flavorful experience.","ingredients":["Shrimp","Lemongrass","Galangal","Kaffir lime leaves","Mushrooms","Fish sauce","Lime juice","Chili paste (Nam Prik Pao)","Fresh chilies","Cilantro"]},{"recipe_name":"Pho Bo","description":"Vietnam's iconic beef noodle soup, renowned for its deeply aromatic and comple

In [4]:
recipes = response.parsed
type(recipes), len(recipes)

(list, 4)

In [5]:
recipes[0]

Recipe(recipe_name='Miso Soup', description="A traditional Japanese soup consisting of a dashi stock into which softened miso paste is mixed. Various optional ingredients may be added depending on the region and season, such as tofu, wakame seaweed, and green onions. It's known for its umami-rich flavor and comforting warmth.", ingredients=['Dashi stock', 'Miso paste', 'Silken tofu', 'Wakame seaweed', 'Green onions'])

In [6]:
recipes[-1].recipe_name, recipes[-1].ingredients

('Laksa',
 ['Coconut milk',
  'Laksa paste',
  'Chicken stock',
  'Rice vermicelli',
  'Shrimp',
  'Chicken breast',
  'Fish cakes',
  'Tofu puffs',
  'Bean sprouts',
  'Hard-boiled egg',
  'Coriander',
  'Lime'])

## Simulate house prices

In [7]:
from typing import Literal

class Home(BaseModel):
    price: int 
    monthly_fee: int 
    living_area: float 
    number_rooms: float
    address: str
    type: Literal["apartment", "house"]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""List 20 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK.""",
    config={"response_mime_type": "application/json", "response_schema": list[Home]},
)

homes = response.parsed
homes[:3]

[Home(price=2500000, monthly_fee=3500, living_area=65.5, number_rooms=2.0, address='Sveavägen 45, Stockholm', type='apartment'),
 Home(price=4800000, monthly_fee=500, living_area=120.0, number_rooms=4.0, address='Villagatan 12, Göteborg', type='house'),
 Home(price=1850000, monthly_fee=2800, living_area=50.0, number_rooms=1.5, address='Storgatan 10, Malmö', type='apartment')]

In [8]:
import pandas as pd 

df = pd.DataFrame([dict(home) for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,address,type
0,2500000,3500,65.5,2.0,"Sveavägen 45, Stockholm",apartment
1,4800000,500,120.0,4.0,"Villagatan 12, Göteborg",house
2,1850000,2800,50.0,1.5,"Storgatan 10, Malmö",apartment
3,3200000,4000,80.0,3.0,"Vasagatan 7, Uppsala",apartment
4,6500000,700,180.0,6.0,"Björkvägen 22, Lund",house


In [9]:
df.query("price < 2_000_000")

,price,monthly_fee,living_area,number_rooms,address,type
2,1850000,2800,50.0,1.5,"Storgatan 10, Malmö",apartment
7,1600000,2500,40.0,1.0,"Drottninggatan 100, Norrköping",apartment
12,1950000,2900,52.0,2.0,"Östra Hamngatan 1, Gävle",apartment
17,1700000,2600,45.0,1.0,"Västra Storgatan 12, Karlstad",apartment
